### Lectura de los archivos csv y conversión a sql con creación de tablas

In [5]:
from Configuración_BD import engine
import pandas as pd

db = engine.connect()

df_alumnos = pd.read_csv("Bases de datos/Alumnos.csv",sep=",") # lee la base de datos "alumnos" que está en formato csv y lo guarda en la variable df
df_profesores = pd.read_csv("Bases de datos/Profesores.csv",sep=",") # lo mismo con "profesores"
df_cursos_profesores = pd.read_csv("Bases de datos/Cursos_profesores.csv",sep=",") # lo mismo con "cursos_profesores"


try:
	df_alumnos.to_sql("alumnos", db, if_exists="fail") # convierte el dataframe (df) de alumnos a sql y crea la tabla profes 
	df_profesores.to_sql("profesores", db, if_exists="fail") # lo mismo con "profesores"
	df_cursos_profesores.to_sql("cursos_profesores", db, if_exists="fail") # lo mismo con "cursos_profesores" 

except ValueError as vx:
	print(vx)
except Exception as ex:
	print (ex)
else:
	print("Tablas creadas con éxito")
finally:	
	db.close()

Tablas creadas con éxito


### Lectura de las tablas y conversión del dataframe a una lista de diccionarios (dataset)

In [6]:
df= pd.read_sql("select * from alumnos", engine) # lee la tabla alumnos (sql) y lo guarda en la variable df
dataset_alumnos=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_alumnos)

db.close()

In [ ]:
df= pd.read_sql("select * from profesores", engine) # lee la tabla profesores (sql) y lo guarda en la variable df
dataset_profesores=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_profesores)


db.close()

In [ ]:
df= pd.read_sql("select * from cursos_profesores", engine) # lee la tabla cursos_profesores (sql) y lo guarda en la variable df
dataset_cursos_profesores=df.to_dict("records")  # transforma la variable df en un diccionario dividido en registros (dataset_cursos_profesores)

db.close()

#### Eliminando el campo "index"

In [ ]:
def eliminar_index(dataset):
    for x in dataset:
        del x["index"]    
    print(dataset)

        
eliminar_index(dataset_alumnos)
eliminar_index(dataset_profesores)

#### Extraer valores únicos de "gender" y traducirlos al español

In [ ]:
def traduccion_gender(dataset):
    lista=[]
    for x in dataset:
        if x["gender"] not in lista:
            lista.append(x["gender"])
    print(lista)

    for x in dataset:
        if x["gender"]=="Female":
            x["gender"]="Femenino"
        elif x["gender"]=="Male":
            x["gender"]="Masculino"
        elif x["gender"]=="Non-binary":
            x["gender"]="No-binario"
        elif x["gender"]=="Polygender":
            x["gender"]="Poligénero"
        elif x["gender"]=="Genderqueer":
            x["gender"]="Género queer"
        elif x["gender"]=="Agender":
            x["gender"]="Agénero"
        elif x["gender"]=="Bigender":
            x["gender"]="Bigénero"
        elif x["gender"]=="Genderfluid":
            x["gender"]="Género fluido"

    print(dataset)

traduccion_gender(dataset_profesores)
traduccion_gender(dataset_alumnos)

#### Convertir los campos personal_id e instructor a número, eliminando los puntos

In [ ]:
def conversion_dni_numero(dataset):
    for x in dataset:
        if "personal_id" in x:                   
            j=x["personal_id"].replace('.','')
            x["personal_id"]=j
        if "instructor" in x:
            j=x["instructor"].replace('.','')
            x["instructor"]=j
    print(dataset)
    
conversion_dni_numero(dataset_cursos_profesores)

#### Eliminar registros repetidos en su totalidad o que tienen un campo repetido 

In [7]:
def eliminar_registros_repetidos_entotalidad(dataset):
    lista_unica = set()   # conjunto con los registros únicos pero en forma de tuplas
    nueva_lista = []    # lista con los registros únicos en forma de diccionarios
    repetidos = []  # lista con los registros eliminados

    for j in dataset:
        t = tuple(j.items()) # convierte a la lista de tuplas de los pares clave-valor de cada registro en una tupla de tuplas
        if t not in lista_unica: # si ese registro en forma de tupla de tuplas no está en el set lo agrega y a su 
                                 # vez agrega el registro en forma de diccionario a la lista
            lista_unica.add(t)
            nueva_lista.append(j)
        else:
            repetidos.append(j) 

    dataset=nueva_lista  
    print(repetidos)

eliminar_registros_repetidos_entotalidad(dataset_alumnos)
    

def eliminar_registros_repetidos_uncampo(dataset,campo):
    valores_unicos = [] # crea una lista con todos los valores únicos del campo seleccionado
    registros_repetidos = [] # crea una lista con todos los registros en los cuales en el campo seleccionado tengan un valor repetido
    for x in dataset:
        for i,j in x.items():      
            if i==campo:          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)        
    for x in dataset:   # se elimina del dataset todos aquellos registros que se encuentren en la lista de registros_repetidos
        if x in registros_repetidos:
            del dataset[dataset.index(x)]   
    print(registros_repetidos) 

eliminar_registros_repetidos_uncampo(dataset_alumnos,"id")   


def buscar_registros_repetidos_uncampo(dataset,campo): # esta función es por si no se quiere eliminar aquellos registros con 
                                                       # campos repetidos, sino que solamente se los quiere encontrar para quizás modificar el 
                                                       # campo antes que borrar todo el registro
    valores_unicos = [] 
    registros_repetidos = [] 
    for x in dataset:
        for i,j in x.items():      
            if i==campo:          
                if j not in valores_unicos:        
                    valores_unicos.append(j)              
                else:                         
                    registros_repetidos.append(x)     
    print(registros_repetidos)  

buscar_registros_repetidos_uncampo(dataset_alumnos,"id") 



[]
[]
[]


#### Para crear las tablas del archivo tablas

In [ ]:
from Tablas import Base,engine,Profesores,Alumnos,Carreras,Profesores_Carreras,Alumnos_Carreras, Facultades,Ramas,Campus,Ubicacion,Genero
db = engine.connect()
Base.metadata.create_all(engine)